**Objectif** : on va dans un premier temps chercher à prédire la consommation électrique uniquement à partir des données historiques. Pour cela on va tronquer la fin de la série, effectuer une prédiction puis vérifier avec les données réelles. Dans un deuxième temps on va utiliser les données météorologiques afin d'améliorer la prédiction.

# 1 - Import des bibliothèques

In [35]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
chemin_scripts = os.path.abspath(os.path.join(os.getcwd(), '..', 'scripts'))
sys.path.append(chemin_scripts)

from fonctions_conso import importation, clean, traitement_valeurs_manquantes

# 2 - Récupération des données

In [20]:
url = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-metropoles-tr/exports/csv?lang=fr&refine=libelle_metropole%3A%22M%C3%A9tropole%20du%20Grand%20Paris%22&facet=facet(name%3D%22libelle_metropole%22%2C%20disjunctive%3Dtrue)&timezone=Europe%2FParis&use_labels=true&delimiter=%3B"
df = traitement_valeurs_manquantes(clean(importation(url)))[['conso_final']]

In [21]:
df.head(2)

,conso_final
date_heure,
2020-01-01 01:00:00,5090.0
2020-01-01 01:15:00,5018.0


# 3 - Test de stationnarité

Tester la stationnarité d'une série temporelle est crucial pour l'analyse et la modélisation des données temporelles. La stationnarité garantit la stabilité des propriétés statistiques de la série au fil du temps, permettant d'appliquer des modèles appropriés et d'obtenir des résultats valides et des prévisions fiables.

On rappelle qu'une série temporelle à valeurs réelles en temps discret $X_1,...,X_t$ est dite faiblement stationnaire si :

$$
\begin{cases}
\mathbb{E}[X_i] = \mu \hspace{1cm} \forall i \\
Var(X_i) = \sigma ^2 \hspace{1cm} \forall i \\
Cov(X_i, X_{i-k}) = \rho _k \hspace{1cm} \forall i, \forall k
\end{cases}
$$

On va commencer par tester la stationnarité de la série des consommations d'électricité de la métropole parisienne afin de choisir par la suite le bon modèle de prédiction. Pour cela, on effectue le test de racine unitaire de Dickey-Füller augmenté (test ADF) dont les hypothèses sont les suivantes :

$$
\begin{cases}
H_0 : \text{présence de racine unitaire} \Leftrightarrow \text{série non stationnaire} \\
H_1 : \text{absence de racine unitaire} \Leftrightarrow \text{série stationnaire}
\end{cases}
$$

Les valeurs critiques du test ADF diffèrent selon l'hypothèse choisie pour le processus générateur de données (DGP) de la série temporelle :

- **DGP 1** (sans partie déterministe, option `regression='n'` dans adfuller) : 
$$ X_t = \phi X_{t-1} + \epsilon_t $$  
Valeur critique au seuil 5% pour le rejet de $H_0$ : -1,95

- **DGP 2** (avec constante, option `regression='c'` dans adfuller) :  
$$ X_t = a + \phi X_{t-1} + \epsilon_t $$  
Valeur critique au seuil 5% pour le rejet de $H_0$ : -2,89

- **DGP 3** (avec constante et trend, option `regression='ct'` dans adfuller) : 
$$ X_t = a + b t + \phi X_{t-1} + \epsilon_t $$  
Valeur critique au seuil 5% pour le rejet de $H_0$ : -3,45

Lorsque la valeur du test ADF est inférieure au seuil critique associé à l'hypothèse choisie, on rejette $H_0$, c’est-à-dire que la série est stationnaire.


In [36]:
def adf_table(series, alpha=0.05):
    results = []
    dgp = {'n' : 'DGP1', 'c' : 'DGP2', 'ct' : 'DGP3'}

    for reg, label in dgp.items():
        adf_result = adfuller(series, regression=reg)

        adf_stat = adf_result[0]
        crit_value = adf_result[4]['5%']

        conclusion = "Stationnaire" if adf_stat < crit_value else "Non stationnaire"

        results.append({
            "DGP": label,
            "ADF statistic": adf_stat,
            "Valeur critique (5%)": crit_value,
            "Conclusion": conclusion
        })

    return pd.DataFrame(results)

In [37]:
df_adf = adf_table(df)

In [38]:
df_adf

,DGP,ADF statistic,Valeur critique (5%),Conclusion
0,DGP1,-1.273610,-1.941001,Non stationnaire
1,DGP2,-5.760323,-2.861554,Stationnaire
2,DGP3,-5.762672,-3.410511,Stationnaire


**Conclusion** : la série est stationnaire autour d'une constante $\neq$ 0. On n'a donc pas besoin de la stationnariser et il va être plus facile d'effectuer des prédictions.

# 4 - Choix du modèle et des paramètres

**Rappels** :
- AR (AutoRegressive) : la valeur actuelle dépend de ses valeurs passées, lorsque la série présente de l'autocorrélation
- MA (MovingAverage) : la valeur actuelle dépend des erreurs passées, lorsque les chocs passés influencent les valeurs futures
- ARMA : AR + MA lorsque les séries sont stationnaires
- ARIMA : extension d'ARMA aux séries non stationnaires basé sur un nombre de différenciations qui rend la série stationnaire
- SARIMA : ARIMA + saisonnalité
- GARCH : modélise la volatilité passée combinée à l'effet des chocs passés

Chaque modèle dépend d'un ou plusieurs paramètres appelés ordre à déterminer à partir des valeurs de la série.

## 4.1 - ARIMA

## 4.2 - SARIMA